In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains, ScrollOrigin
import pandas as pd
from time import sleep

#### Click Iphone

In [24]:
url = 'https://www.thegioididong.com/dtdd'
driver = webdriver.Chrome()
driver.get(url)

#### Click Iphone, Samsung, Xiaomi

In [25]:
hang_elm = driver.find_element(By.XPATH, '/html/body/div[6]/div[4]/section/div[1]/div/div[3]/div[1]')
hang_elm.click()

In [26]:
samsung_elm = driver.find_element(By.XPATH, '/html/body/div[6]/div[4]/section/div[1]/div/div[3]/div[2]/div[1]/a[1]')
iphone_elm = driver.find_element(By.XPATH, '/html/body/div[6]/div[4]/section/div[1]/div/div[3]/div[2]/div[1]/a[2]')
xiaomi_elm = driver.find_element(By.XPATH, '/html/body/div[6]/div[4]/section/div[1]/div/div[3]/div[2]/div[1]/a[4]')


In [27]:
samsung_elm.click()
iphone_elm.click()
xiaomi_elm.click()

In [28]:
xem_ket_qua_elm = driver.find_element(By.XPATH, '/html/body/div[6]/div[4]/section/div[1]/div/div[3]/div[2]/div[2]/a[2]')
xem_ket_qua_elm.click()
sleep(2)

#### Show more:

In [29]:
show_more_elm = driver.find_element(By.XPATH, '//*[@id="categoryPage"]/div[3]/div[2]/a')

In [30]:
while True:
    try:
        show_more_elm.click()
        sleep(2)
    except:
        print('we are at the end of the page!')
        break

we are at the end of the page!


#### Get product infomation

In [31]:
# products_elm = driver.find_elements(By.CLASS_NAME, 'item')
# len(products_elm)

In [32]:
listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')
len(product_list)

84

##### Create get product info function

In [33]:
def get_product_info(product_elm):
    # opt1: get phone name:
    phone_name = product_elm.find_element(By.XPATH, './/h3[@title]').text

    # get price:
    price = product_elm.find_element(By.XPATH, value='.//strong[@class="price"]').text

    # opt1: get utility:
    utility = product_elm.find_element(By.CLASS_NAME, 'utility').text

    # get product image:
    phone_img_link = product_elm.find_element(By.XPATH, './/img').get_attribute('src')

    result = {
        'name': phone_name,
        'price': price,
        'utility': utility,
        'img_link': phone_img_link
    }
    return result

#### Get all products infomation

In [34]:
all_product_list = []

for prd_elm in product_list:
    product_info = get_product_info(product_elm=prd_elm)
    all_product_list.append(product_info)

In [35]:
df = pd.DataFrame(all_product_list)
df.head()

,name,price,utility,img_link
0,iPhone 15 Pro Max 256GB,29.590.000₫,Chip Apple A17 Pro 6 nhân\nRAM: 8 GB\nDung lượ...,https://cdn.tgdd.vn/Products/Images/42/305658/...
1,iPhone 13 256GB,16.990.000₫,Chip Apple A15 Bionic\nRAM: 4 GB\nDung lượng: ...,https://cdn.tgdd.vn/Products/Images/42/250258/...
2,Samsung Galaxy S23 Ultra 5G 256GB,24.990.000₫,Chip Snapdragon 8 Gen 2 for Galaxy\nRAM: 8 GB\...,https://cdn.tgdd.vn/Products/Images/42/249948/...
3,Xiaomi Redmi Note 13 (8GB/128GB),4.390.000₫,Chip Snapdragon 685 8 nhân\nRAM: 8 GB\nDung lư...,https://cdn.tgdd.vn/Products/Images/42/309831/...
4,Samsung Galaxy A55 5G (8GB/256GB),10.190.000₫,Chip Exynos 1480 8 nhân\nRAM: 8 GB\nDung lượng...,https://cdn.tgdd.vn/Products/Images/42/322670/...


#### Get Product Description

In [36]:
def get_product_description(product_elm):
    # B1: Click vao product:
    product_elm.click()

    sleep(2)

    # B2: get production description:
    try:
        product_description = driver.find_element(By.CLASS_NAME, 'content-article').get_attribute('textContent')
    except:
        product_description = driver.find_element(By.CLASS_NAME, 'article__content').get_attribute('textContent')

    # B3: Back lại trang trước:
    driver.back()
    sleep(2)

    return product_description

In [37]:
product_list_url = driver.current_url
product_list_url

'https://www.thegioididong.com/dtdd#c=42&m=2,80,2235&o=17&pi=4'

In [38]:
all_product_list = []
for i in range(len(product_list)):
    for t in range(10):
        try:
            prd_elm = product_list[i]
            product_info = get_product_info(prd_elm)
            product_info['description'] = get_product_description(prd_elm)
            all_product_list.append(product_info)
            break
        except:
            driver.get(product_list_url)
            sleep(2)
            listproduct_elm = driver.find_element(By.CLASS_NAME, 'listproduct')
            product_list = listproduct_elm.find_elements(By.CLASS_NAME, 'item')


In [39]:
desc_df = pd.DataFrame( all_product_list)
df = pd.merge(
    left = df,
    right = desc_df[['name', 'description']],
    how = 'left',
    on = 'name'
)

In [42]:
now = str(pd.Timestamp('now')).replace(':', '')
now

'2024-05-18 232104.429392'

In [43]:
# save data:
df.to_excel(fr'C:\Users\Windows\Desktop\Crawl data from website\data\phone_data {now}.xlsx', index=False)